# Zodiac Race

## Initial setup

In [1]:
#Import the relevant libraries
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Read the files

First, we import the csv file and determine the column that contains 'ABCDE'

In [2]:
#Import csv files
z_file = "./The_Zodiac_Race_Challenge_Dataset.csv"
z_df = pd.read_csv(z_file)

In [3]:
z_df.head()

,Desert,Mountain,Water,Land,Forest,Snow,Average_Temperature,Is_Raining,Had_A_Good_Sleep,Winner,Winner_Time,Second,Second_Time,Third,Third_Time
0,7376,4387,7769,7374,814,2677,18,Yes,Yes,rat,218,dog,242,dragon,245
1,7911,5771,4993,1855,9605,283,18,Yes,Yes,tiger,216,snake,224,rat,239
2,8948,9631,1855,4913,3531,3746,39,Yes,No,ox,191,rabbit,217,horse,240
3,1735,6643,6066,9976,9119,9912,24,Yes,No,goat,319,rooster,322,horse,334
4,2001,8287,4062,8763,9751,2820,26,Yes,Yes,horse,255,goat,259,rooster,263


In [4]:
# Find column that contains'ABCDE'
bad_cols = [col for col in z_df.columns if 'ABCDE' in z_df[col].values]
print(bad_cols)

['Had_A_Good_Sleep']


C:\Users\Ivan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


In [5]:
# Remove bad col
z_df.drop(bad_cols, axis=1, inplace=True)

In [6]:
# Separate input variables (X) from output variables (Y) 

X = z_df.iloc[:, :8]
Y = z_df.iloc[:, 8:]

In [7]:
X.head()

,Desert,Mountain,Water,Land,Forest,Snow,Average_Temperature,Is_Raining
0,7376,4387,7769,7374,814,2677,18,Yes
1,7911,5771,4993,1855,9605,283,18,Yes
2,8948,9631,1855,4913,3531,3746,39,Yes
3,1735,6643,6066,9976,9119,9912,24,Yes
4,2001,8287,4062,8763,9751,2820,26,Yes


In [8]:
Y.head()

,Winner,Winner_Time,Second,Second_Time,Third,Third_Time
0,rat,218,dog,242,dragon,245
1,tiger,216,snake,224,rat,239
2,ox,191,rabbit,217,horse,240
3,goat,319,rooster,322,horse,334
4,horse,255,goat,259,rooster,263


In [9]:
animal_list = ['rat', 'ox', 'tiger', 'rabbit', 'dragon', 'snake', 
               'horse', 'goat', 'monkey', 'rooster', 'dog', 'pig']

top_3 = ['Winner', 'Second', 'Third']

#Generate df (one-hot encoded) indicating whether each animal (column) finishes in the top 3

Y_top3 = pd.DataFrame({animal:z_df[top_3].isin([animal]).apply(sum, axis=1) for animal in animal_list}).astype(int)

In [10]:
Y_top3.head()

,rat,ox,tiger,rabbit,dragon,snake,horse,goat,monkey,rooster,dog,pig
0,1,0,0,0,1,0,0,0,0,0,1,0
1,1,0,1,0,0,1,0,0,0,0,0,0
2,0,1,0,1,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,1,0,1,0,0
4,0,0,0,0,0,0,1,1,0,1,0,0


# Data preprocessing

## Engineer Features

In [11]:
# Total distance:
X.eval('Total_distance = Desert + Mountain + Water + Land + Forest + Snow', inplace= True)
# Convert Yes/No to 1/0
X['Is_Raining'] = X['Is_Raining'].map({'Yes':1, 'No':0}) 

# scale data
X_scaled = (X - X.min()) / (X.max() - X.min())
# X_scaled = (X - X.mean())/X.std()

In [12]:
from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
import copy

In [13]:
Y.head()

,Winner,Winner_Time,Second,Second_Time,Third,Third_Time
0,rat,218,dog,242,dragon,245
1,tiger,216,snake,224,rat,239
2,ox,191,rabbit,217,horse,240
3,goat,319,rooster,322,horse,334
4,horse,255,goat,259,rooster,263


In [14]:
def to_time_df(finisher_col, time_col): 
    return pd.get_dummies(finisher_col).apply( lambda x: x*time_col)

time_map = {'Winner': 'Winner_Time',
            'Second': 'Second_Time',
            'Third': 'Third_Time'
           }

Y_time = sum([to_time_df(Y[pos_name], Y[pos_time]) for (pos_name, pos_time )in time_map.items()])[animal_list]

assert sum((Y_time > 0).sum(axis=1) != 3) ==0
Y_time.head()

,dog,dragon,goat,horse,monkey,ox,pig,rabbit,rat,rooster,snake,tiger
0,242,245,0,0,0,0,0,0,218,0,0,0
1,0,0,0,0,0,0,0,0,239,0,224,216
2,0,0,0,240,0,191,0,217,0,0,0,0
3,0,0,319,334,0,0,0,0,0,322,0,0
4,0,0,259,255,0,0,0,0,0,263,0,0


## 1st stage:
### Predict whether each animal finishes in top 3

In [27]:
#split into training and test sets
X_train, X_test, Y_top3_train, Y_top3_test, Y_time_train, Y_time_test = train_test_split(X_scaled, Y_top3.astype(int), Y_time, random_state=0)

In [16]:
classifiers = [
    KNeighborsClassifier(3),
#     SVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(random_state=0),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
#     QuadraticDiscriminantAnalysis(),
    LogisticRegression()
    ]


clf_names = [clf.__class__.__name__ for clf in classifiers]

In [17]:
refit_clfs = True

def helper(clf, animal, X_train, Y_top3_train):
    
    print('fitting {} to {} \n'.format(clf.__class__.__name__, animal))
    return clf.fit(X_train, Y_top3_train[animal])


# fit classifiers
if refit_clfs==True:
    clfs_fitted_df = pd.DataFrame([[helper(clf, animal, X_train, Y_top3_train) for clf in copy.deepcopy(classifiers)] 
                                    for animal in animal_list],
                                  columns= clf_names,
                                  index = animal_list)
    pickle.dump(clfs_fitted_df, open("clfs_fitted_df", 'wb') )

# use cached copy
else:
    clfs_fitted_df= pickle.load(open("clfs_fitted_df", 'rb'))


fitting KNeighborsClassifier to rat 

fitting DecisionTreeClassifier to rat 

fitting RandomForestClassifier to rat 



C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


fitting AdaBoostClassifier to rat 

fitting GradientBoostingClassifier to rat 

fitting GaussianNB to rat 

fitting LinearDiscriminantAnalysis to rat 

fitting LogisticRegression to rat 

fitting KNeighborsClassifier to ox 

fitting DecisionTreeClassifier to ox 



C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


fitting RandomForestClassifier to ox 



...

C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


fitting RandomForestClassifier to pig 



C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


fitting AdaBoostClassifier to pig 

fitting GradientBoostingClassifier to pig 

fitting GaussianNB to pig 

fitting LinearDiscriminantAnalysis to pig 

fitting LogisticRegression to pig 



C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [18]:
def acc_top3(clf_fitted, X_test, y_test):
    
#get accurracy of predicting top3 finishes (how well does classifier predict a true "top 3" label?)
#leaving out the majority of instances without a "top3" label
#clf: FITTED classifier

    y_test_intop3 = y_test[y_test==1]
    X_test_intop3 = X_test[y_test==1]
    
    y_pred_intop3= clf_fitted.predict(X_test_intop3)
    
    return y_pred_intop3.mean()

In [19]:
def compare_clfs(animal):

#given name of animal, generates dataframe comparing classifiers 

    return pd.DataFrame([(clf_fitted.__class__.__name__
                          ,clf_fitted.score(X_test, Y_top3_test[animal]) #get score for each fitted classifier
#                           ,acc_top3(clf_fitted, X_test, Y_top3_test[animal])
                         ) for clf_fitted in clfs_fitted_df.loc[animal]]
                           ,columns=['classifier', 'acc']).set_index('classifier')

In [20]:
clf_all_animals = pd.concat([compare_clfs(animal) for animal in animal_list], axis=1, keys=animal_list)
# clf_all_animals.reindex((df.columns), axis=1)
clf_all_animals

,rat,ox,tiger,rabbit,dragon,snake,horse,goat,monkey,rooster,dog,pig
,acc,acc,acc,acc,acc,acc,acc,acc,acc,acc,acc,acc
classifier,,,,,,,,,,,,
KNeighborsClassifier,0.8972,0.9364,0.9082,0.9409,0.9247,0.8777,0.9061,0.9166,0.9053,0.9475,0.9036,0.9273
DecisionTreeClassifier,0.9223,0.9832,0.9186,0.9785,0.9237,0.9081,0.9198,0.9256,0.9184,0.9579,0.9282,0.9591
RandomForestClassifier,0.9456,0.9868,0.9421,0.9854,0.9484,0.9266,0.9465,0.9460,0.9438,0.9688,0.9483,0.9711
AdaBoostClassifier,0.9242,0.9783,0.9356,0.9786,0.9273,0.8959,0.9112,0.9298,0.8920,0.9634,0.9238,0.9546
GradientBoostingClassifier,0.9450,0.9851,0.9423,0.9852,0.9550,0.9241,0.9459,0.9493,0.9340,0.9695,0.9453,0.9667
GaussianNB,0.8443,0.9502,0.8654,0.9471,0.8960,0.8157,0.8630,0.8792,0.8481,0.9496,0.8519,0.8820
LinearDiscriminantAnalysis,0.7804,0.9445,0.8348,0.9379,0.9125,0.7187,0.8676,0.9000,0.8527,0.9441,0.7933,0.8690
LogisticRegression,0.7794,0.9467,0.8354,0.9408,0.9123,0.7190,0.8683,0.9002,0.8533,0.9504,0.7932,0.8683


### Compare classifiers

In [21]:
# %%debug

# clf_all_animals=clf_all_animals.swaplevel(axis=1)
# clf_all_animals.reindex(sorted(clf_all_animals.columns), axis=1)
best_clfs = clf_all_animals.idxmax()

name_clf_dict = {clf.__class__.__name__:clf for clf in classifiers}
best_clfs.value_counts()

best_clf = name_clf_dict[best_clfs.mode().loc[0]]
print('best clf: {}'.format(best_clf.__class__.__name__))

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(5)<module>()

ipdb> c
best clf: RandomForestClassifier


## Fit Regressors

In [22]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LogisticRegression

In [23]:
regressors = [
#               GradientBoostingRegressor(),
              GradientBoostingRegressor(n_estimators=1000, max_depth=5),
              RandomForestRegressor(), 
              AdaBoostRegressor(),
              LogisticRegression()
             ]

rgg_names = [rgg.__class__.__name__ for rgg in regressors]

In [24]:
refit_rggs = True

def helper_rgg(rgg, animal, X_train, Y_time_train):
    
    X_selected = X_train[~Y_time_train[animal].isnull()]
    y_selected = Y_time_train[animal][~Y_time_train[animal].isnull()]

    print('fitting {} to {} \n'.format(rgg.__class__.__name__, animal))
    return rgg.fit(X_selected, y_selected)

if refit_rggs==True:
    rggs_fitted_df = pd.DataFrame([[helper_rgg(rgg, animal, X_train, Y_time_train) for rgg in copy.deepcopy(regressors)] 
                                    for animal in animal_list]
                                   ,columns= rgg_names
                                   ,index = animal_list)
    pickle.dump(rggs_fitted_df, open("rggs_fitted_df", 'wb') )

else:
    rggs_fitted_df= pickle.load(open("rggs_fitted_df", 'rb'))


fitting GradientBoostingRegressor to rat 

fitting RandomForestRegressor to rat 



C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


fitting AdaBoostRegressor to rat 

fitting LogisticRegression to rat 



C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


fitting GradientBoostingRegressor to ox 

fitting RandomForestRegressor to ox 



...

C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


fitting GradientBoostingRegressor to pig 

fitting RandomForestRegressor to pig 



C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


fitting AdaBoostRegressor to pig 

fitting LogisticRegression to pig 



C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [28]:
def compare_rggs(animal):

#given name of animal, generates dataframe comparing classifiers 

    return pd.DataFrame([(rgg_fitted.__class__.__name__
                          ,rgg_fitted.score(X_train[~Y_time_train[animal].isnull()], 
                                            Y_time_train[animal][~Y_time_train[animal].isnull()])
                         ) for rgg_fitted in rggs_fitted_df.loc[animal]]
                           ,columns=['regressor', 'acc']).set_index('regressor')

In [ ]:
# %%debug

rgg_all_animals = pd.concat([compare_rggs(animal) for animal in animal_list], axis=1, keys=animal_list)

In [45]:
rgg_all_animals

,rat,ox,tiger,rabbit,dragon,snake,horse,goat,monkey,rooster,dog,pig
,acc,acc,acc,acc,acc,acc,acc,acc,acc,acc,acc,acc
regressor,,,,,,,,,,,,
GradientBoostingRegressor,0.899308,0.993515,0.884592,0.990295,0.902111,0.901922,0.902795,0.888012,0.907156,0.950537,0.906027,0.956540
RandomForestRegressor,0.965375,0.988200,0.958914,0.984254,0.961700,0.962936,0.963602,0.957911,0.961921,0.957449,0.964159,0.971506
AdaBoostRegressor,0.269329,0.795938,0.318708,0.776510,0.445660,0.412963,0.394074,0.324897,0.393482,0.389590,0.329705,0.292060
LogisticRegression,0.678500,0.808233,0.763667,0.822967,0.771767,0.678500,0.665600,0.722167,0.684900,0.889300,0.729200,0.788500


In [29]:
best_rggs = rgg_all_animals.idxmax()

name_rgg_dict = {rgg.__class__.__name__:rgg for rgg in regressors}
best_rggs.value_counts()

best_rgg = name_rgg_dict[best_rggs.mode().loc[0]]
print('best rgg: {}'.format(best_rgg.__class__.__name__))

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(3)<module>()

ipdb> n
> <string>(4)<module>()

ipdb> n
> <string>(6)<module>()

ipdb> p best_rggs
rat      acc        RandomForestRegressor
ox       acc    GradientBoostingRegressor
tiger    acc        RandomForestRegressor
rabbit   acc    GradientBoostingRegressor
dragon   acc        RandomForestRegressor
snake    acc        RandomForestRegressor
horse    acc        RandomForestRegressor
goat     acc        RandomForestRegressor
monkey   acc        RandomForestRegressor
rooster  acc        RandomForestRegressor
dog      acc        RandomForestRegressor
pig      acc        RandomForestRegressor
dtype: object
ipdb> p type(best_rggs)
<class 'pandas.core.series.Series'>
ipdb> c
best rgg: RandomForestRegressor


## Predict top3 using regression
### Compare results vs using classifer technique to predict top 3

In [31]:
def pred_time(X, best_rgg):
# rgg_fitted_series: series of fitted (single) regressor for all all animals 
# returns dataframe of predicted timings for all animals given input data X 
    return pd.DataFrame([rggs_fitted_df.loc[animal,best_rgg.__class__.__name__].predict(X) for animal in animal_list], 
                        index=animal_list,
                        columns=X.index
                       ).T
    

def pred_top3_rgg(X, best_rgg):
# returns dataframe Y of (one-hot) indicator of whether animal finished in top3
    return pred_time(X_test, best_rgg).apply(lambda row: row.nsmallest(3) ,axis=1).notnull().reindex(animal_list, axis=1)
    

def pred_top3_clf(X, best_clf):
# returns dataframe Y of (one-hot) indicator of whether animal finished in top3
    return pd.DataFrame([clfs_fitted_df.loc[animal,best_clf.__class__.__name__]
                         .predict(X) for animal in animal_list],
                        index=animal_list,
                        columns=X.index
                       ).T

In [46]:
pred_top3_clf_df = pred_top3_clf(X_test, best_clf)
pred_top3_rgg_df= pred_top3_rgg(X_test,best_rgg)

In [50]:
print("Accuracy in predicting whether animal finishes in top3 \nUsing Regression: {} \nUsing Classification: {}" \
      .format((Y_top3_test == pred_top3_rgg_df).mean().mean(),(Y_top3_test == pred_top3_clf_df).mean().mean())
     )

Accuracy in predicting whether animal finishes in top3 
Using Regression: 0.5013166666666666 
Using Classification: 0.9549499999999999


In [655]:
# Predict top 3 finihers in order using

# Method 1
# Pure regression: regression, then sort by timings

# Method 2
# Classification to pick animals predicted to finish in top3, then sort by timings 
# If < 3 finishers are predicted to be in top3, choose next fastest timing from animal not predicted in top3

In [656]:
#Generate test rank set, entries where animals are tied

Y_rank_test = Y_time_test.rank(axis=1)
X_rank_test = X_test

In [657]:
# # Remove entries where 2 or more of the top3 finishers are tied
# tied = Y_rank_test.mode(axis=1)[2].isnull()

# Y_rank_test = Y_rank_test[~tied]
# X_rank_test = X_test[~tied]

In [658]:
#Method 1

#Dataframe of predicting rankings using regression
pred_rank_rgg = pred_time(X_rank_test, best_rgg).rank(axis=1).apply(lambda row: row.nsmallest(3) ,axis=1).reindex(animal_list, axis=1)

#number of positions predicted correctly for each entry
print((pred_rank_rgg==Y_rank_test).sum(axis=1).value_counts())

#proportion of times where all top3 rankings where predicted correctly
rank_acc_rgg = ((pred_rank_rgg==Y_rank_test).sum(axis=1)==3).mean()

0    5552
1    2086
3    1358
2    1004
dtype: int64


In [659]:
#Method 2

def pred_rank_m2(X, best_rgg, best_clf):
    pred_rank = pred_time(X, best_rgg).rank(axis=1)
    pred_top3_clf_df = pred_top3_clf(X, best_clf)
    
    #ranking(using timings from rgg) of animals predicted by clf to finish in top 3 
    intop3_ranked = pred_rank[pred_top3_clf_df.astype(bool)]
    outtop3_ranked = pred_rank[~pred_top3_clf_df.astype(bool)] + 12
    
    return  intop3_ranked.fillna(outtop3_ranked).apply(lambda row: row.nsmallest(3) ,axis=1) \
            .reindex(animal_list, axis=1).rank(axis=1)


In [660]:
pred_rank_m2_df = pred_rank_m2(X_rank_test, best_rgg, best_clf)

In [661]:
print((pred_rank_m2_df==Y_rank_test).sum(axis=1).value_counts())

3    3971
1    2953
2    2087
0     989
dtype: int64


In [662]:
rank_acc_m2 = ((pred_rank_m2_df==Y_rank_test).sum(axis=1)==3).mean()

In [674]:
clfs_fitted_df = pd.DataFrame([[helper(clf, animal, X_scaled, Y_top3) for clf in copy.deepcopy([best_clf])] 
                                for animal in animal_list],
                              columns= [best_clf.__class__.__name__],
                              index = animal_list)

rggs_fitted_df = pd.DataFrame([[helper_rgg(rgg, animal, X_scaled, Y_time) for rgg in copy.deepcopy([best_rgg])] 
                                for animal in animal_list]
                               ,columns= [best_rgg.__class__.__name__]
                               ,index = animal_list)



fitting RandomForestClassifier to rat 

fitting RandomForestClassifier to ox 

fitting RandomForestClassifier to tiger 

fitting RandomForestClassifier to rabbit 

fitting RandomForestClassifier to dragon 

fitting RandomForestClassifier to snake 

fitting RandomForestClassifier to horse 

fitting RandomForestClassifier to goat 

fitting RandomForestClassifier to monkey 

fitting RandomForestClassifier to rooster 

fitting RandomForestClassifier to dog 

fitting RandomForestClassifier to pig 

fitting GradientBoostingRegressor to rat 

fitting GradientBoostingRegressor to ox 

fitting GradientBoostingRegressor to tiger 

fitting GradientBoostingRegressor to rabbit 

fitting GradientBoostingRegressor to dragon 

fitting GradientBoostingRegressor to snake 

fitting GradientBoostingRegressor to horse 

fitting GradientBoostingRegressor to goat 

fitting GradientBoostingRegressor to monkey 

fitting GradientBoostingRegressor to rooster 

fitting GradientBoostingRegressor to dog 

fitting G

In [676]:
X_submit = pd.DataFrame([[2345, 8764, 6689, 7332, 1050, 3741, 40]], columns=X_test.columns[:7])
X_submit['Is_Raining']=0
X_submit['Total_distance']= X_submit.loc[::7].sum(axis=1)

pred_rank_m2(X_submit, best_rgg, best_clf)

,rat,ox,tiger,rabbit,dragon,snake,horse,goat,monkey,rooster,dog,pig
0,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0


In [700]:
a = [i for i in range(10)]
b=a


In [701]:
b=b + [2]

In [703]:
a

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]